<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 01 - SQL Server Master Instance Queries

In this tutorial you will learn how to run standard SQL Server Queries against the Master Instance (MI) in a SQL Server big data cluster. 

We'll start with a simple set of queries to explore the Instance: 



In [1]:
/* Instance Version */
SELECT @@VERSION; 
GO

/* General Configuration */
USE master;  
GO  
EXEC sp_configure;
GO

/* Databases on this Instance */
SELECT db.name AS 'Database Name'
, Physical_Name AS 'Location on Disk'
, Cast(Cast(Round(cast(mf.size as decimal) * 8.0/1024000.0,2) as decimal(18,2)) as nvarchar) 'Size (GB)'
FROM sys.master_files mf
INNER JOIN 
    sys.databases db ON db.database_id = mf.database_id
WHERE mf.type_desc = 'ROWS';
GO

SELECT * from sys.master_files


(1 row affected)

Total execution time: 00:00:00.1399588

Commands completed successfully.

Total execution time: 00:00:00.0121076

(No column name)
Microsoft SQL Server 2019 (CTP2.4) - 15.0.1400.75 (X64) Mar 16 2019 11:53:26 Copyright (C) 2019 Microsoft Corporation Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>


...

(14 rows affected)

Total execution time: 00:00:00.0405191

name,minimum,maximum,config_value,run_value
allow polybase export,0,1,0,0
allow updates,0,1,0,0
backup checksum default,0,1,0,0
backup compression default,0,1,0,0
clr enabled,0,1,1,1
column encryption enclave type,0,1,0,0
contained database authentication,0,1,0,0
cross db ownership chaining,0,1,0,0
default language,0,9999,0,0
external scripts enabled,0,1,0,0


Database Name,Location on Disk,Size (GB)
master,/var/opt/mssql/data/master.mdf,0.00
tempdb,/var/opt/mssql/data/tempdb.mdf,0.01
model,/var/opt/mssql/data/model.mdf,0.01
msdb,/var/opt/mssql/data/MSDBData.mdf,0.01
DWDiagnostics,/var/opt/mssql/data/DWDiagnostics.mdf,0.50
DWConfiguration,/var/opt/mssql/data/DWConfiguration.mdf,0.01
DWQueue,/var/opt/mssql/data/DWQueue.mdf,0.01


database_id,file_id,file_guid,type,type_desc,data_space_id,name,physical_name,state,state_desc,size,max_size,growth,is_media_read_only,is_read_only,is_sparse,is_percent_growth,is_name_reserved,is_persistent_log_buffer,create_lsn,drop_lsn,read_only_lsn,read_write_lsn,differential_base_lsn,differential_base_guid,differential_base_time,redo_start_lsn,redo_start_fork_guid,redo_target_lsn,redo_target_fork_guid,backup_lsn,credential_id
1,1,NULL,0,ROWS,1,master,/var/opt/mssql/data/master.mdf,0,ONLINE,512,-1,10,0,0,0,1,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,2,NULL,1,LOG,0,mastlog,/var/opt/mssql/data/mastlog.ldf,0,ONLINE,256,-1,10,0,0,0,1,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,1,NULL,0,ROWS,1,tempdev,/var/opt/mssql/data/tempdb.mdf,0,ONLINE,1024,-1,8192,0,0,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,2,NULL,1,LOG,0,templog,/var/opt/mssql/data/templog.ldf,0,ONLINE,1024,-1,8192,0,0,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,1,NULL,0,ROWS,1,modeldev,/var/opt/mssql/data/model.mdf,0,ONLINE,1024,-1,8192,0,0,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,2,NULL,1,LOG,0,modellog,/var/opt/mssql/data/modellog.ldf,0,ONLINE,1024,-1,8192,0,0,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,1,8fcf842e-a342-414e-a894-f45196a9db85,0,ROWS,1,MSDBData,/var/opt/mssql/data/MSDBData.mdf,0,ONLINE,1712,-1,10,0,0,0,1,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,2,2f169600-e65f-41b4-b9f0-348866a58fe8,1,LOG,0,MSDBLog,/var/opt/mssql/data/MSDBLog.ldf,0,ONLINE,64,268435456,10,0,0,0,1,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
5,1,bd6a15ac-29cf-40ed-ab8b-0f4617f5b39b,0,ROWS,1,DWDiagnostics,/var/opt/mssql/data/DWDiagnostics.mdf,0,ONLINE,64000,256000,8192,0,0,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
5,2,0a35d259-b590-4338-9ebe-04b814c522f4,1,LOG,0,DWDiagnostics_log,/var/opt/mssql/data/DWDiagnostics_log.ldf,0,ONLINE,9216,268435456,8192,0,0,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


## Ingest data into the SQL Server Databases

Before we start working with data, we need to bring it in to the system. We have several options to do that, from the `bcp` utility to SQL Server Integration Services, the Azure Data Factory and more. 

For the structured data, we'll use the SQL Server `RESTORE` command to bring in two databases from the location we specified earlier with the `kubectl` command.

The Code below shows all of that: 

In [1]:
/* Add the Customer Databases for Wide World Importers */
USE [master]
RESTORE DATABASE [WideWorldImporters] 
FROM  DISK = N'/var/opt/mssql/data/WWI.bak' 
WITH  FILE = 1
,  REPLACE
,  MOVE N'WWI_Primary' TO N'/var/opt/mssql/data/WideWorldImporters.mdf'
,  MOVE N'WWI_UserData' TO N'/var/opt/mssql/data/WideWorldImporters_UserData.ndf'
,  MOVE N'WWI_Log' TO N'/var/opt/mssql/data/WideWorldImporters.ldf'
,  MOVE N'WWI_InMemory_Data_1' TO N'/var/opt/mssql/data/WideWorldImporters_InMemory_Data_1'
,  NOUNLOAD,  STATS = 5;
GO


## Query Data

The SQL Server 2019 big data cluster Master Instance is a SQL Server Instance - and as such has most all of the query facilities and capabilities of Microsoft SQL Server running on Linux.

**TODO:** Run some standard queries. Investigate simple ML.

In [2]:
/* Get some general information about the data in the WWI OLTP system */
USE WideWorldImporters;
GO

/* Show the Populations. 
Where do we have the most people?
 */
SELECT TOP 10 CityName as 'City Name'
, StateProvinceName as 'State or Province'
, sp.LatestRecordedPopulation as 'Population'
, CountryName
FROM Application.Cities AS city
JOIN Application.StateProvinces AS sp on
    city.StateProvinceID = sp.StateProvinceID
JOIN Application.Countries AS ctry on 
    sp.CountryID=ctry.CountryID


Commands completed successfully.

Total execution time: 00:00:00.0777492

(10 rows affected)

Total execution time: 00:00:00.2088759

City Name,State or Province,Population,CountryName
Abbeville,Georgia,9992167,United States
Acree,Georgia,9992167,United States
Acworth,Georgia,9992167,United States
Adairsville,Georgia,9992167,United States
Adel,Georgia,9992167,United States
Adgateville,Georgia,9992167,United States
Adrian,Georgia,9992167,United States
Ailey,Georgia,9992167,United States
Alamo,Georgia,9992167,United States
Alapaha,Georgia,9992167,United States


In [4]:
/* Show Customer Sales in WWI OLTP
Where do we have the most customers?
*/
USE WideWorldImporters;
GO

SELECT TOP 10 s.CustomerID
, s.CustomerName
, sc.CustomerCategoryName
,  pp.FullName AS PrimaryContact
,  ap.FullName AS AlternateContact
,  s.PhoneNumber
,  s.FaxNumber
,  bg.BuyingGroupName
,  s.WebsiteURL
,  dm.DeliveryMethodName AS DeliveryMethod
,  c.CityName AS CityName
,  s.DeliveryLocation AS DeliveryLocation
,  s.DeliveryRun
,  s.RunPosition
FROM Sales.Customers AS s
    LEFT OUTER JOIN Sales.CustomerCategories AS sc
    ON s.CustomerCategoryID = sc.CustomerCategoryID
    LEFT OUTER JOIN [Application].People AS pp
    ON s.PrimaryContactPersonID = pp.PersonID
    LEFT OUTER JOIN [Application].People AS ap
    ON s.AlternateContactPersonID = ap.PersonID
    LEFT OUTER JOIN Sales.BuyingGroups AS bg
    ON s.BuyingGroupID = bg.BuyingGroupID
    LEFT OUTER JOIN [Application].DeliveryMethods AS dm
    ON s.DeliveryMethodID = dm.DeliveryMethodID
    LEFT OUTER JOIN [Application].Cities AS c
    ON s.DeliveryCityID = c.CityID

Commands completed successfully.

Total execution time: 00:00:00.0772630

(10 rows affected)

Total execution time: 00:00:00.0879768

CustomerID,CustomerName,CustomerCategoryName,PrimaryContact,AlternateContact,PhoneNumber,FaxNumber,BuyingGroupName,WebsiteURL,DeliveryMethod,CityName,DeliveryLocation,DeliveryRun,RunPosition
1,Tailspin Toys (Head Office),Novelty Shop,Waldemar Fisar,Laimonis Berzins,(308) 555-0100,(308) 555-0101,Tailspin Toys,http://www.tailspintoys.com,Delivery Van,Lisco,0xE6100000010CE73F5A52A4BF444010638852B1A759C0,,
2,"Tailspin Toys (Sylvanite, MT)",Novelty Shop,Lorena Cindric,Hung Van Groesen,(406) 555-0100,(406) 555-0101,Tailspin Toys,http://www.tailspintoys.com/Sylvanite,Delivery Van,Sylvanite,0xE6100000010CF37A8BE2B05B4840056FA35CF5F75CC0,,
3,"Tailspin Toys (Peeples Valley, AZ)",Novelty Shop,Bhaargav Rambhatla,Mudar Jevtic,(480) 555-0100,(480) 555-0101,Tailspin Toys,http://www.tailspintoys.com/PeeplesValley,Delivery Van,Peeples Valley,0xE6100000010CC97553CA6B2241404FACF82B892E5CC0,,
6,"Tailspin Toys (Jessie, ND)",Novelty Shop,Biswajeet Thakur,Radha Shah,(701) 555-0100,(701) 555-0101,Tailspin Toys,http://www.tailspintoys.com/Jessie,Delivery Van,Jessie,0xE6100000010CB9A6406667C54740BD7E77E13D8F58C0,,
7,"Tailspin Toys (Frankewing, TN)",Novelty Shop,Kalidas Nadar,Filip Nedvidek,(423) 555-0100,(423) 555-0101,Tailspin Toys,http://www.tailspintoys.com/Frankewing,Delivery Van,Frankewing,0xE6100000010C8DEE2076A6984140C8EBC1A478B655C0,,
8,"Tailspin Toys (Bow Mar, CO)",Novelty Shop,Kanti Kotadia,Hoa Cu,(303) 555-0100,(303) 555-0101,Tailspin Toys,http://www.tailspintoys.com/BowMar,Delivery Van,Bow Mar,0xE6100000010CEC3191D26CD043407BB482F032435AC0,,
9,"Tailspin Toys (Netcong, NJ)",Novelty Shop,Sointu Aalto,Jae-Gon Min,(201) 555-0100,(201) 555-0101,Tailspin Toys,http://www.tailspintoys.com/Netcong,Delivery Van,Netcong,0xE6100000010CCD80690712734440EFBB332438AD52C0,,
10,"Tailspin Toys (Wimbledon, ND)",Novelty Shop,Siddhartha Parkar,Paula Matos,(701) 555-0100,(701) 555-0101,Tailspin Toys,http://www.tailspintoys.com/Wimbledon,Delivery Van,Wimbledon,0xE6100000010CEF14BC32B9954740CEE737A76D9D58C0,,
11,"Tailspin Toys (Devault, PA)",Novelty Shop,Elnaz Javan,Jayashish Ghatak,(215) 555-0100,(215) 555-0101,Tailspin Toys,http://www.tailspintoys.com/Devault,Delivery Van,Devault,0xE6100000010C18343EDD8A094440681888AF40E252C0,,
12,"Tailspin Toys (Biscay, MN)",Novelty Shop,Heloisa Fernandes,Amornrat Rattanaporn,(218) 555-0100,(218) 555-0101,Tailspin Toys,http://www.tailspintoys.com/Biscay,Delivery Van,Biscay,0xE6100000010C4D158C4AEA69464040232246999157C0,,


## Next Step: Data Virtualization

**TODO:** Add Link